<a href="https://colab.research.google.com/github/SRB2005450/Doc_Query_System/blob/main/queryproj0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task: Simplified document query system

In [98]:
!pip install -U streamlit langchain-community langchain cohere pypdf pyngrok

!pip install faiss-cpu


In [99]:
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import Cohere
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import tempfile
import os
import json

# Set up Cohere API Key
COHERE_API_KEY = "HOCzstxWNIY3I0vdNnNS8CVPoRYyuWlLwYJQDGxz"

# Initialize session state for history
if "history" not in st.session_state:
    st.session_state.history = []

def load_document(file):
    """Loads text from uploaded documents."""
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file.read())
        tmp_file_path = tmp_file.name

    if file.name.endswith(".pdf"):
        loader = PyPDFLoader(tmp_file_path)
    else:
        loader = TextLoader(tmp_file_path)

    documents = loader.load()
    os.remove(tmp_file_path)
    return documents

def process_documents(docs):
    """Splits and indexes the documents."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(docs)
    embeddings = CohereEmbeddings(
        model="embed-english-v2.0",
        cohere_api_key=COHERE_API_KEY,
        user_agent="langchain-cohere"
    )
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store

def get_answer(query, vector_store):
    """Retrieves answers based on user query."""
    llm = Cohere(cohere_api_key=COHERE_API_KEY)
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
    return qa_chain.run(query)

def export_to_pdf(history):
    """Exports query history to a PDF file."""
    temp_pdf_path = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf").name
    c = canvas.Canvas(temp_pdf_path, pagesize=letter)
    width, height = letter
    c.drawString(100, height - 40, "Document Query System - Query History")

    y_position = height - 80
    for i, entry in enumerate(history):
        c.drawString(100, y_position, f"{i+1}. Q: {entry['query']}")
        y_position -= 20
        c.drawString(120, y_position, f"A: {entry['answer']}")
        y_position -= 30
        if y_position < 50:
            c.showPage()
            y_position = height - 80

    c.save()
    return temp_pdf_path

# Streamlit UI
st.title("📄 Document Query System")

uploaded_file = st.file_uploader("Upload a document (PDF/TXT)", type=["txt", "pdf"])

if uploaded_file:
    st.success(f"✅ Upload completed: **{uploaded_file.name}**")
    documents = load_document(uploaded_file)
    vector_store = process_documents(documents)

    query = st.text_input("Enter your query:")
    if st.button("Get Answer"):
        if query:
            answer = get_answer(query, vector_store)
            st.session_state.history.append({"query": query, "answer": answer})
            st.write("**Answer:**", answer)
        else:
            st.warning("⚠️ Please enter a query.")

    if st.session_state.history:
        st.subheader("Query History")
        for i, entry in enumerate(st.session_state.history[::-1]):
            st.write(f"{i+1}. **Q:** {entry['query']}\n   **A:** {entry['answer']}")

        pdf_path = export_to_pdf(st.session_state.history)
        with open(pdf_path, "rb") as pdf_file:
            st.download_button("📄 Download Query History (PDF)", pdf_file, file_name="query_history.pdf", mime="application/pdf")


2025-03-31 09:03:01.375 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.382 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.383 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.384 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.385 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:03:01.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [100]:
!pip install -U pyngrok
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "2v2PgEpPWfVeR7UaO70OApgYMvX_6UncvVrwQtfeLHyVfc3r8"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [101]:
!streamlit run app.py &>/dev/null &


In [102]:
public_url = ngrok.connect(8501, "http")
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://bdf5-34-136-207-202.ngrok-free.app" -> "http://localhost:8501"
